In [2]:
#1
%matplotlib inline
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
from consts import sensor_day_seconds as day_second
from consts import sensor_idd as idd
from consts import sensor_idds_uniq as idds_uniq
from consts import sensor_idds_uniq_sorted as idds_uniq_sorted
from consts import sensor_a_temperature as a_temperature
from consts import sensor_s_temperature as s_temperature
from consts import sensor_unix_column as unix_column
from consts import sensor_idd as idd
epoch_length = 30
import multiprocessing as mp
from multiprocessing import Pool
import time

In [3]:
#2
input_file_new = open('3_dict_list_new.pkl','rb')
input_list_new = pickle.load(input_file_new)
input_file_new.close()

In [4]:
#3
# 将epoch epoch_len epoch_time 都flat化
len_list_new = []
time_stamp_list_new = []
epoch_list_new = []
for start_time in input_list_new:
    # print(len(start_time[0]))
    for dict in start_time[0]:
        len_list_new.append(len(dict))
        epoch_list_new.append(dict)
    start_point = start_time[1]
    time_stamps = np.arange(start_point,start_point+day_second+epoch_length,epoch_length)
    time_stamp_list_new.extend(time_stamps[:-1])
len_array_new = np.array(len_list_new)
time_stamp_array_new = np.array(time_stamp_list_new) # 时间序列
epoch_array_new = np.array(epoch_list_new)

ipykernel_launcher:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
start_list = np.arange(1162393768,1178747998+day_second,day_second)
ddf_list_new = []
for start_point in start_list:
    time_stamps = np.arange(start_point,start_point+day_second+epoch_length,epoch_length)
    sub_df_list = []
    df = pd.read_hdf('day2.h5',str(start_point))
    for i in range(len(time_stamps)-1):
        sub_df = df[(df[unix_column] > time_stamps[i]) & (df[unix_column] <= time_stamps[i+1])]
        sub_df_list.append(sub_df)
    ddf_list_new.extend(sub_df_list)

In [ ]:
# 修改为多线程化读入

In [4]:
#4
def read_sub_df(start_point):
    time_stamps = np.arange(start_point,start_point+day_second+epoch_length,epoch_length)
    sub_df_list = []
    df = pd.read_hdf('day2_sa.h5',str(start_point))
    for i in range(len(time_stamps)-1):
        sub_df = df[(df[unix_column] > time_stamps[i]) & (df[unix_column] <= time_stamps[i+1])]
        
        sub_df_list.append(sub_df)
    print("Done \t {}".format(start_point))
    return (start_point,sub_df_list)

In [5]:
#5
start_list = np.arange(1162393768,1178747998+day_second,day_second)
manager = mp.Manager()
# aggreateData = manager.dict()
p = Pool(16)
tmp_result = []

for start_point in start_list:
    result = p.apply_async(func=read_sub_df,args=(start_point,))
    tmp_result.append(result)
p.close()
p.join()


Done 	 1162825768
Done 	 1162739368
Done 	 1162652968
Done 	 1162480168
Done 	 1162566568
Done 	 1162912168
Done 	 1163171368
Done 	 1162393768
Done 	 1163084968
Done 	 1162998568
Done 	 1163344168
Done 	 1163430568
Done 	 1163257768Done 	 1163603368

Done 	 1163516968
Done 	 1163689768
Done 	 1163862568
Done 	 1163776168
Done 	 1163948968
Done 	 1164035368
Done 	 1164121768
Done 	 1164208168
Done 	 1164294568
Done 	 1164380968
Done 	 1164467368
Done 	 1164553768
Done 	 1164640168
Done 	 1164726568
Done 	 1164812968
Done 	 1164899368
Done 	 1164985768
Done 	 1165072168
Done 	 1165158568
Done 	 1165244968
Done 	 1165331368
Done 	 1165417768
Done 	 1165504168
Done 	 1165590568
Done 	 1165676968
Done 	 1165763368
Done 	 1165849768
Done 	 1165936168
Done 	 1166022568
Done 	 1166108968
Done 	 1166195368
Done 	 1166281768
Done 	 1166368168
Done 	 1166454568
Done 	 1166540968
Done 	 1166627368
Done 	 1166713768
Done 	 1166800168
Done 	 1166886568
Done 	 1166972968
Done 	 1167059368
Done 	 116

In [6]:
#6
origin_time = start_list[0]

results = []
for i in start_list:
    results.append([])

for item in tmp_result:
    temp = item.get()
    results[int((temp[0]-origin_time)/86400)].extend(temp[1])

ddf_list_new = []
for item in results:
    ddf_list_new.extend(item)

In [7]:
#7
index1 = np.where(len_array_new > 40)[0]
maybe_array_new = epoch_array_new[index1]
print(len(maybe_array_new))

473651


In [8]:
#8
# init maybe_dict 
maybe_dict = {}
for i in idds_uniq:
    maybe_dict[i] = 0


for item in maybe_array_new:
    uniq_item = np.unique(item)
    for j in uniq_item:
        maybe_dict[j] += 1
print(maybe_dict)

{7: 125799, 8: 471452, 11: 458757, 14: 472821, 17: 458104, 19: 445912, 23: 465393, 26: 464479, 27: 84998, 31: 433210, 33: 459727, 34: 472749, 35: 473617, 40: 455014, 46: 466469, 47: 129074, 49: 451279, 51: 444594, 55: 151859, 57: 251149, 59: 266859, 60: 429722, 61: 259182, 62: 315350, 63: 247709, 65: 353682, 69: 240795, 70: 466843, 71: 246772, 72: 467454, 73: 464136, 75: 375903, 76: 413628, 79: 456291, 80: 425423, 81: 379307, 82: 187571, 84: 437322, 87: 450836, 88: 356917, 89: 447802, 93: 450445, 96: 460671, 97: 468705, 98: 328718, 100: 452027, 103: 442609, 104: 444617, 109: 234528, 111: 362542, 121: 473651}


In [10]:
#9                                                      
maybe_dict_sorted = sorted(maybe_dict.items(),key=lambda kv:(kv[1],kv[0]),reverse=True)
maybe_dict_sorted = np.array(maybe_dict_sorted)

In [11]:
sub_sensor_list = maybe_dict_sorted.T[0]
print(sub_sensor_list)

[121  35  14  34   8  97  72  70  46  23  26  73  96  33  11  17  79  40
 100  49  87  93  89  19 104  51 103  84  31  60  80  76  81  75 111  88
  65  98  62  59  61  57  63  71  69 109  82  55  47   7  27]


In [5]:
#10
# 测试sensor数量为多少合适
def get_sub_index(sensor_count):
    sub_sensor_list = idds_uniq_sorted[:sensor_count]
    sub_index = []
    for i, item in enumerate(epoch_array_new):
        uniq_item = np.unique(item)
        diff = np.setdiff1d(sub_sensor_list,uniq_item)
        if len(diff) == 0:
            sub_index.append(i)
    return len(sub_index),sub_index

In [32]:
epoch_array_new.shape

(550080,)

In [30]:
for i,item in enumerate(sub_index_array):
    print("id",i+10,"number:",item[0])

id 10 number: 378683
id 11 number: 378487
id 12 number: 348561
id 13 number: 345626
id 14 number: 344499
id 15 number: 343160
id 16 number: 330523
id 17 number: 327771
id 18 number: 327440
id 19 number: 311179
id 20 number: 278013
id 21 number: 272067
id 22 number: 272067
id 23 number: 262523
id 24 number: 262258
id 25 number: 240924
id 26 number: 240666
id 27 number: 223298
id 28 number: 209368
id 29 number: 192740
id 30 number: 183545
id 31 number: 177827
id 32 number: 177825
id 33 number: 172067
id 34 number: 171416
id 35 number: 170802
id 36 number: 137159
id 37 number: 112691
id 38 number: 104700
id 39 number: 65497
id 40 number: 63813
id 41 number: 49961
id 42 number: 46459
id 43 number: 9169
id 44 number: 9112
id 45 number: 9101
id 46 number: 9026
id 47 number: 4103
id 48 number: 887
id 49 number: 0
id 50 number: 0


In [28]:
len(sub_index_array)

41

In [6]:
#11 多进程版 
start_time = time.time()
def multi_get_sub_index(i):
    return get_sub_index(i)

sensor_count_list = np.linspace(10,50,41,dtype=int)

manager = mp.Manager()
# aggreateData = manager.dict()
p = Pool(16)
tmp_result = []

for i in sensor_count_list:
    result = p.apply_async(func=multi_get_sub_index,args=(i,))
    tmp_result.append(result)
p.close()
p.join()
print(time.time()-start_time)

78.22364854812622


In [7]:
#11-1
sub_index_list = []
for item in tmp_result:
    sub_index_list.append(item.get())
sub_index_array = np.array(sub_index_list)

ipykernel_launcher:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [23]:
sub_index_array.shape

(41, 2)

In [34]:
# obsolete 12 
def get_length_count(window):
    for item in sub_index_array:
        zeros = np.zeros_like(epoch_array)
        zeros[item[1]] = 1
        sum = 0
        for j in range(len(zeros)):
            tmp = zeros[j:j+window]
            if 0 not in tmp:
                sum += 1
        print(sum)

In [18]:
# 12-1 DUMP
output_file = open('sub_index_array_drop_na_sa.pkl','wb')
pickle.dump(sub_index_array,output_file)
output_file.close()

In [ ]:
# 12-2 LOAD
input_file = open('sub_index_array_drop_na_sa.pkl','rb')
sub_index_array = pickle.load(input_file)
input_file.close()

In [25]:
#13
def get_data_matrixs(sensor_count,window,sub_index_array = None):
    print(idds_uniq_sorted[:sensor_count])
    tmp = sub_index_array[sensor_count-10][1]
    zeros = np.zeros_like(epoch_array_new)
    zeros[tmp] = 1

    sum_list = []
    for i in range(len(zeros)):
        tmp1 = zeros[i:i+window]
        if 0 not in tmp1:
            sum_list.append(i)
            
    return sum_list
indices = get_data_matrixs(19,7,sub_index_array)
print(len(indices))

[51, 103, 34, 70, 87, 46, 104, 96, 11, 17, 8, 76, 97, 23, 26, 84, 49, 73, 100]


In [17]:
len(indices)

226922

In [23]:
indices[5]

95141

In [ ]:
#14
# 丢弃多余帧
def drop_useless_sensor(df,sensor_list):
    # print(sensor_list)
    for i in df[idd].values:
        if i not in sensor_list:
            df = df[df[idd] != i]
    # print(len(df))
    if len(df) < len(sensor_list):
        return None
    return df
# t = drop_useless_sensor(ddf_list[indices[3]],idds_uniq_sorted[:19])
# print(len(t))

In [ ]:
# ignore
def handle_df(ddf_list,indice,sensor_list):
    df = ddf_list[indice]
    df = df.drop_duplicates(idd)
    df = drop_useless_sensor(df,sensor_list)
    return df

df_sample = handle_df(ddf_list,indices[0],idds_uniq_sorted[:19])

In [ ]:
# before 15
for i, df in enumerate(ddf_list):
    with open('./data/{}.pkl'.format(i),'wb') as f:
        pickle.dump(df,f)

In [ ]:
#15 multi_processing
def multi_convert_df_to_dm(indice,sensor_list,window=7):
    df_list = []
    for i in range(window):
        with open('./data/{}.pkl'.format(indice+i),'rb') as f:
            sub_df = pickle.load(f)
        df_list.append(sub_df)
    return_list = []
    for df in df_list:
        df = df.drop_duplicates(idd)
        df = drop_useless_sensor(df,sensor_list)
        if type(df) == type(None):
            break
        df = df.sort_values(by=idd)
        flat_matrix = np.zeros(100)
        indice_dict = {63:10,23:14,17:15,47:20,7:21,27:22,26:24,14:25,121:26,46:27,8:28,111:32,75:33,57:34,59:35,40:36,34:37,61:41,71:42,65:43,82:45,62:46,76:48,51:51,11:52,70:53,49:54,60:57,84:60,81:61,55:62,72:63,73:64,35:66,31:67,69:68,79:69,103:70,80:71,96:72,97:75,100:76,33:77,19:78,93:79,109:80,87:81,104:84,89:91,88:92,98:94}
        return_list.append(df[s_temperature].values)
    return np.array(return_list).T
res_array = multi_convert_df_to_dm(indices[46],idds_uniq_sorted[:19],7)
print(res_array)

In [ ]:
#15
def convert_df_to_dm(ddf_list,indice,sensor_list,window=7):
    # print(sensor_list)
#     print(len(sensor_list))
    df_list = []
    for i in range(window):
        df_list.append(ddf_list[indice+i])
#     df = ddf_list[indice]
    return_list = []
    for df in df_list:
        df = df.drop_duplicates(idd)
#         print(indice,type(df))
#         print(len(df))
        df = drop_useless_sensor(df,sensor_list)
        if type(df) == type(None):
            break
        df = df.sort_values(by=idd)
        flat_matrix = np.zeros(100)
        indice_dict = {63:10,23:14,17:15,47:20,7:21,27:22,26:24,14:25,121:26,46:27,8:28,111:32,75:33,57:34,59:35,40:36,34:37,61:41,71:42,65:43,82:45,62:46,76:48,51:51,11:52,70:53,49:54,60:57,84:60,81:61,55:62,72:63,73:64,35:66,31:67,69:68,79:69,103:70,80:71,96:72,97:75,100:76,33:77,19:78,93:79,109:80,87:81,104:84,89:91,88:92,98:94}
        return_list.append(df[s_temperature].values)
#         for i,indice in enumerate(df[idd].values):
#             flat_matrix[indice_dict[indice]] = df[s_temperature].values[i]
    return np.array(return_list).T
    

res_array = convert_df_to_dm(ddf_list,indices[45],idds_uniq_sorted[:19],7)
print(res_array)

In [ ]:
for i,df in enumerate(ddf_list):
    print(i)

In [ ]:
res_array

In [ ]:
#16多线程
array_list2 = []
manager = mp.Manager()

p = Pool(16)

start_time = time.time()
def multi_get_sub_index(i):
    return get_sub_index(i)

tmp_result = []

for i in indices:
    result = p.apply_async(func=multi_convert_df_to_dm,args=(i,idds_uniq_sorted[:19],7,))
    tmp_result.append(result)
p.close()
p.join()

for item in tmp_result:
    array_list2.append(item.get())

print(time.time()-start_time)

In [ ]:
#16
start_time = time.time()
array_list2 = []
for i in indices:
    array_list2.append(convert_df_to_dm(ddf_list,i,idds_uniq_sorted[:19],7))
print(time.time()-start_time)

In [ ]:
#before 17-4
sum = 0
new_indices = []
for i,item in enumerate(array_list2):
    if np.isnan(item).any():
        print(i)
#         sum+=1
    else:
        new_indices.append(i)

In [ ]:
time_stamp_array

In [ ]:
#17-1
with open('4_tensor.pkl','wb') as f:
    pickle.dump(array_list2,f)

In [ ]:
#17-2
with open('4_time_stamp_array.pkl','wb') as f:
    pickle.dump(time_stamp_array,f)

In [ ]:
#17-3
with open('4_indices_na.pkl','wb') as f:
    pickle.dump(indices,f)

In [ ]:
#17-4
with open('4_indices_no_na.pkl','wb') as f:
    pickle.dump(new_indices,f)